In [22]:
import pandas as pd

from src.data_processor import DataProcessor

# Load data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')
for date in df['Date'].unique():
    week_data = df[df['Date'] == date]

In [ ]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))



# notebooks/03_trading_simulation.ipynb

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from src.models.trading_engine import DynamicTradingEngine
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import tensorflow as tf
from tensorflow.keras.models import Sequential



# Load data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')


# First, load and train models for each household

# Train prediction models
# household_predictors = {}
# for household in df['HouseholdID'].unique():
#     household_df = df[df['HouseholdID'] == household].copy()
#     X, y = processor.prepare_training_data(household_df)
    
#     # Split data
#     train_size = int(len(X) * 0.8)
#     X_train = X[:train_size]
#     y_train = y[:train_size]
    
#     # Train model
#     predictor = ExpenditurePredictor()
#     predictor.train(X_train, y_train)
#     household_predictors[household] = predictor
household_models = {}
for household in df['HouseholdID'].unique():
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household] = tf.keras.models.load_model(model_path)

def simulate_future_trading(current_date, trained_models):
    """Simulate trading based on predicted values using pre-trained models"""
    future_trades = []
    
    for household in df['HouseholdID'].unique():
        # Get recent data for prediction
        recent_data = df[
            (df['HouseholdID'] == household) & 
            (df['Date'] <= current_date)
        ].tail(4)  # Last 4 weeks for prediction
        
        # Prepare input for prediction using the modified prepare_prediction_data method
        X_pred = processor.prepare_prediction_data(recent_data)
        
        # Use the pre-trained model for this household
        predictor = trained_models[household]
        predicted_expenditure = predictor.predict(X_pred)
        
        # Use last known values for usage and generation
        predicted_usage = recent_data['EnergyUsed'].mean()
        predicted_generation = recent_data['EnergyGeneratedFromRenewableSources'].mean()
        
        future_trades.append({
            'HouseholdID': household,
            'PredictedUsage': predicted_usage,
            'PredictedGeneration': predicted_generation,
            'PredictedExpenditure': predicted_expenditure[0]
        })
    
    return pd.DataFrame(future_trades)




# Initialize trading engine
engine = DynamicTradingEngine()

def get_season(date):
    month = pd.to_datetime(date).month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

def simulate_weekly_trading(week_data, time_of_day=14, trained_models=household_models):
    # Use predicted values instead of historical data
    current_date = week_data['Date'].iloc[0]
    future_data = simulate_future_trading(current_date, trained_models)
    
    # Calculate predicted surplus/deficit
    future_data['Delta'] = future_data['PredictedGeneration'] - future_data['PredictedUsage']
    
    # Rest of the function remains similar but uses predicted values
    sellers = future_data[future_data['Delta'] > 0].copy()
    buyers = future_data[future_data['Delta'] < 0].copy()
    
    print(f"Time: {time_of_day}:00")
    print(f"Number of sellers: {len(sellers)}")
    print(f"Number of buyers: {len(buyers)}")
    
    if len(sellers) == 0 or len(buyers) == 0:
        return pd.DataFrame()
    
    # Get season for the trading day
    season = get_season(week_data['Date'].iloc[0])
    
    # Calculate dynamic price based on current market conditions
    current_market_price = engine.calculate_dynamic_price(
        sellers['Delta'].sum(),
        -buyers['Delta'].sum(),
        time_of_day,
        season
    )
    
    print(f"Current market price: ${current_market_price:.4f}/kWh")
    
    # Prepare sellers dataframe with more competitive rates
    sellers_df = pd.DataFrame({
        'householdID': sellers['HouseholdID'],
        'unitsToBeSold': sellers['Delta'],
        'sellingRate': current_market_price * 0.9  # More competitive selling rate
    })
    
    # Prepare buyers dataframe with higher buying rates
    buyers_df = pd.DataFrame({
        'householdID': buyers['HouseholdID'],
        'demand': -buyers['Delta'],
        'buyingRate': current_market_price * 1.1  # Higher buying rate to ensure trades
    })
    
    return engine.match_orders(sellers_df, buyers_df, time_of_day, season)

# Simulate trading for each week at different times
trading_times = [21]  # 9 AM, 2 PM, 7 PM
weekly_trades = []

for date in df['Date'].unique():
    week_data = df[df['Date'] == date]
    for time in trading_times:
        trades = simulate_weekly_trading(week_data, time)
        if not trades.empty:
            trades['Date'] = date
            trades['TimeOfDay'] = time
            weekly_trades.append(trades)
if not weekly_trades:
    print("No trades were generated during the simulation")
    trading_history = pd.DataFrame(columns=['buyer_id', 'seller_id', 'amount',
                                          'price_per_unit', 'total_cost',
                                          'timestamp', 'Date', 'TimeOfDay'])
else:
    trading_history = pd.concat(weekly_trades, ignore_index=True)

# Visualizations
# 1. Trading Volume Over Time with Time-of-Day Breakdown
fig1 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_volume = time_data.groupby('Date')['amount'].sum()
    fig1.add_trace(go.Scatter(
        x=daily_volume.index,
        y=daily_volume.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig1.update_layout(
    title='Trading Volume by Time of Day',
    xaxis_title='Date',
    yaxis_title='Energy Traded (kWh)'
)
fig1.show()

# 2. Price Trends by Time of Day
fig2 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_price = time_data.groupby('Date')['price_per_unit'].mean()
    fig2.add_trace(go.Scatter(
        x=daily_price.index,
        y=daily_price.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig2.update_layout(
    title='Average Trading Price by Time of Day',
    xaxis_title='Date',
    yaxis_title='Price ($/kWh)'
)
fig2.show()

# 3. Seasonal Trading Analysis
trading_history['Season'] = trading_history['Date'].apply(get_season)
seasonal_stats = trading_history.groupby('Season').agg({
    'amount': 'sum',
    'price_per_unit': 'mean',
    'total_cost': 'sum'
}).round(2)

print("\nSeasonal Trading Statistics:")
print(seasonal_stats)

# 4. Household Trading Summary
household_summary = pd.DataFrame({
    'Total_Sales': trading_history.groupby('seller_id')['total_cost'].sum(),
    'Total_Purchases': trading_history.groupby('buyer_id')['total_cost'].sum(),
    'Volume_Sold': trading_history.groupby('seller_id')['amount'].sum(),
    'Volume_Bought': trading_history.groupby('buyer_id')['amount'].sum()
}).fillna(0).round(2)

print("\nHousehold Trading Summary:")
print(household_summary)

# 5. Trading Network Visualization (Last Day)
def plot_trading_network(trades_sample):
    network_fig = go.Figure()
    
    for _, trade in trades_sample.iterrows():
        network_fig.add_trace(go.Scatter(
            x=[trade['seller_id'], trade['buyer_id']],
            y=[trade['amount'], trade['amount']],
            mode='lines+markers',
            name=f"${trade['total_cost']:.2f}",
            text=[f"Seller: {trade['seller_id']}<br>Price: ${trade['price_per_unit']:.3f}/kWh", 
                  f"Buyer: {trade['buyer_id']}<br>Amount: {trade['amount']} kWh"],
            hoverinfo='text'
        ))
    
    network_fig.update_layout(
        title=f'Energy Trading Network (Time: {trades_sample["TimeOfDay"].iloc[0]}:00)',
        showlegend=False
    )
    return network_fig

last_date = trading_history['Date'].max()
for time in trading_times:
    last_trades = trading_history[
        (trading_history['Date'] == last_date) & 
        (trading_history['TimeOfDay'] == time)
    ]
    if not last_trades.empty:
        network_fig = plot_trading_network(last_trades)
        network_fig.show()


In [11]:
def predict_future_weeks(df, household_models, num_weeks=10):
    """Predict energy patterns and trading positions for next n weeks"""
    last_date = pd.to_datetime(df['Date'].max())
    future_dates = [last_date + pd.Timedelta(weeks=i+1) for i in range(num_weeks)]
    
    future_predictions = []
    
    for household in df['HouseholdID'].unique():
        # Get last 4 weeks of data for prediction
        recent_data = df[df['HouseholdID'] == household]
        X_pred = processor.prepare_prediction_data(recent_data)
        
        # Get predictions for this household
        predictor = household_models[household]
        
        # Use recent averages for baseline predictions
        base_usage = recent_data['EnergyUsed'].mean()
        base_generation = recent_data['EnergyGeneratedFromRenewableSources'].mean()
        
        for future_date in future_dates:
            # Adjust predictions based on season
            season = get_season(future_date)
            season_mult = {
                'summer': 1.4,
                'winter': 0.6,
                'spring': 1.0,
                'fall': 0.8
            }[season]
            
            predicted_generation = base_generation * season_mult
            predicted_usage = base_usage
            
            # Calculate if household will be buyer or seller
            delta = predicted_generation - predicted_usage
            trading_position = 'Seller' if delta > 0 else 'Buyer'
            
            # Calculate dynamic price based on time of day (using peak hour 14:00)
            market_price = engine.calculate_dynamic_price(
                total_supply=predicted_generation,
                total_demand=predicted_usage,
                time_of_day=14,
                season=season
            )
            
            future_predictions.append({
                'Date': future_date,
                'HouseholdID': household,
                'PredictedUsage': round(predicted_usage, 2),
                'PredictedGeneration': round(predicted_generation, 2),
                'TradingPosition': trading_position,
                'DynamicPrice': round(market_price, 4)
            })
    
    return pd.DataFrame(future_predictions)

# Generate predictions for next 10 weeks
future_df = predict_future_weeks(df, household_models)

# Display predictions grouped by date
print("\nTrading Forecast for Next 10 Weeks:")
# print(future_df['TradingPosition'].value_counts())
print(future_df)



Trading Forecast for Next 10 Weeks:
         Date HouseholdID  PredictedUsage  PredictedGeneration  \
0  2025-02-09          H1          945.15               342.88   
1  2025-02-16          H1          945.15               342.88   
2  2025-02-23          H1          945.15               342.88   
3  2025-03-02          H1          945.15               571.46   
4  2025-03-09          H1          945.15               571.46   
5  2025-03-16          H1          945.15               571.46   
6  2025-03-23          H1          945.15               571.46   
7  2025-03-30          H1          945.15               571.46   
8  2025-04-06          H1          945.15               571.46   
9  2025-04-13          H1          945.15               571.46   
10 2025-02-09          H2         1435.22               452.93   
11 2025-02-16          H2         1435.22               452.93   
12 2025-02-23          H2         1435.22               452.93   
13 2025-03-02          H2         1435.

In [13]:
future_df.to_csv('../data/future_trading_forecast.csv', index=False)